In [96]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
from pylab import rcParams
sns.set_style("darkgrid")
import json
import requests
from pandas.io.json import json_normalize


import warnings
warnings.filterwarnings("ignore")
warnings.warn("This will not show")

%matplotlib inline

pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows',2000)
#pd.set_option('display.max_info_rows', 10)
#pd.set_option('display.max_colwidth', 100)
#pd.set_option('display.max_info_columns', 5000)
#pd.set_option('display.expand_frame_repr', True)
#pd.set_option('display.width', 10)

In [97]:
df = pd.read_csv('scout_car_cleaned_data.csv')
df.T




,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,...,15844,15845,15846,15847,15848,15849,15850,15851,15852,15853,15854,15855,15856,15857,15858,15859,15860,15861,15862,15863,15864,15865,15866,15867,15868,15869,15870,15871,15872,15873,15874,15875,15876,15877,15878,15879,15880,15881,15882,15883,15884,15885,15886,15887,15888,15889,15890,15891,15892,15893,15894,15895,15896,15897,15898,15899,15900,15901,15902,15903,15904,15905,15906,15907,15908,15909,15910,15911,15912,15913,15914,15915,15916,15917,15918
make_model,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,Audi A1,...,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace,Renault Espace
body_type,Sedans,Sedans,Sedans,Sedans,Sedans,Sedans,Station wagon,Compact,Sedans,Sedans,Sedans,Sedans,Sedans,Sedans,Sedans,Sedans,Sedans,Sedans,Compact,Sedans,Sedans,Sedans,Sedans,Sedans,Sedans,Sedans,Compact,Sedans,Sedans,Compact,Compact,Sedans,Compact,Sedans,Sedans,Compact,Sedans,Sedans,Sedans,Compact,Compact,Compact,Compact,Compact,Sedans,Sedans,Sedans,Sedans,Compact,Compact,Compact,Sedans,Sedans,Compact,Compact,Compact,Compact,Sedans,Sedans,Sedans,Sedans,Compact,Sedans,Sedans,Compact,Compact,Sedans,Sedans,Sedans,Sedans,Sedans,Compact,Sedans,Sedans,Compact,...,Van,Van,Van,Van,Van,Van,Van,Station wagon,Van,Van,Van,Van,Van,Van,Van,Station wagon,Van,Van,Van,Van,Van,Station wagon,Van,Van,Van,Van,Van,Van,Van,Van,Van,Van,Van,Van,Van,Van,Sedans,Van,Van,Van,Station wagon,Van,Van,Station wagon,Van,Station wagon,Station wagon,Van,Van,Van,Van,Station wagon,Van,Van,Van,Van,Van,Van,Sedans,Van,Station wagon,Compact,Van,Van,Van,Van,Van,Van,Van,Van,Van,Van,Van,Van,Van
price,15770,14500,14640,14500,16790,15090,16422,14480,16700,17990,18500,17990,18380,18399,17990,18700,18900,18500,18400,18500,18500,18500,18900,18850,18500,8999,11444,11300,18900,11290,11290,11440,11445,11700,11900,11949,11890,11600,11900,11630,11880,11800,11850,11569,11490,11490,11500,11790,11490,11470,11480,11180,11450,11100,10900,10999,10900,10950,10800,10600,10800,9950,10300,10000,10490,18900,18900,18900,18900,18890,18890,18890,18880,18880,18880,...,46666,46660,46490,45990,46130,45990,45890,45490,44999,43950,43911,43899,43890,43990,43800,43790,43545,43490,43490,43490,43315,43490,43080,49490,49690,49100,48990,

In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15919 entries, 0 to 15918
Data columns (total 33 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   make_model           15919 non-null  object 
 1   body_type            15859 non-null  object 
 2   price                15919 non-null  int64  
 3   vat                  11406 non-null  object 
 4   km                   14895 non-null  float64
 5   Type                 15917 non-null  object 
 6   Warranty             4853 non-null   float64
 7   Body Color           15322 non-null  object 
 8   Paint Type           10147 non-null  object 
 9   Nr. of Doors         15707 non-null  float64
 10  Gearing Type         15919 non-null  object 
 11  Cylinders            10239 non-null  float64
 12  Fuel                 15919 non-null  object 
 13  Comfort_Convenience  14999 non-null  object 
 14  Entertainment_Media  14545 non-null  object 
 15  Extras               12957 non-null 

In [99]:
df.isnull().sum()/df.shape[0]*100

make_model              0.000000
body_type               0.376908
price                   0.000000
vat                    28.349771
km                      6.432565
Type                    0.012564
Warranty               69.514417
Body Color              3.750236
Paint Type             36.258559
Nr. of Doors            1.331742
Gearing Type            0.000000
Cylinders              35.680633
Fuel                    0.000000
Comfort_Convenience     5.779257
Entertainment_Media     8.631195
Extras                 18.606696
Safety_Security         6.168729
Gears                  29.599849
age                    10.032037
Previous_Owners        41.711163
hp_kW                   0.552799
Inspection_new         75.299956
Upholstery_type        30.598656
Upholstery_color       31.898989
Nr_of_Seats             6.137320
Displacement_cc         3.115774
Weight_kg              43.809285
Drive_chain            43.080596
Cons_Comb              12.770903
Cons_City              15.302469
Cons_Count

In [100]:
df.shape

(15919, 33)

In [101]:
def fill_most(df, group_col, col_name):
    
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        mode = list(df[cond][col_name].mode())
        if mode != []:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[cond][col_name].mode()[0])
        else:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[col_name].mode()[0])

In [102]:
def fill_prop(df, group_col, col_name):
    
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        df.loc[cond, col_name] = df.loc[cond, col_name].fillna(method="ffill").fillna(method="bfill")
    df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")

In [103]:
def fill(df, group_col1, group_col2, col_name, method): 
    if method == "mode":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                mode1 = list(df[cond1][col_name].mode())
                mode2 = list(df[cond2][col_name].mode())
                if mode2 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].mode()[0])
                elif mode1 != []:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond1][col_name].mode()[0])
                else:
                    df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[col_name].mode()[0])
                
    elif method == "median":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(df[cond2][col_name].median()).fillna(df[cond1][col_name].median()).fillna(df[col_name].median())
                
    elif method == "ffill":           
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                df.loc[cond2, col_name] = df.loc[cond2, col_name].fillna(method="ffill").fillna(method="bfill")
                
        for group1 in list(df[group_col1].unique()):
            cond1 = df[group_col1]==group1
            df.loc[cond1, col_name] = df.loc[cond1, col_name].fillna(method="ffill").fillna(method="bfill")            
           
        df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")

# 2. searchıng each rows

# age

In [104]:
df["age"].value_counts(dropna=False)

1.0    4522
3.0    3674
2.0    3273
0.0    2853
NaN    1597
Name: age, dtype: int64

In [105]:
df['age'].fillna('?', inplace=True)

In [106]:
df["age"].value_counts(dropna=False)

1.0    4522
3.0    3674
2.0    3273
0.0    2853
?      1597
Name: age, dtype: int64

In [107]:
df.groupby("age")["km"].describe()

,count,mean,std,min,25%,50%,75%,max
age,,,,,,,,
0.0,2706.0,2085.355137,5365.880693,1.0,10.00,50.0,3000.0,127022.0
1.0,4484.0,18035.239072,11052.524314,1.0,9990.00,17872.0,25078.5,136000.0
2.0,3272.0,41754.940709,28295.747886,1.0,21541.75,34752.0,54805.5,317000.0
3.0,3674.0,77442.520958,39170.142977,10.0,48000.00,72914.5,99950.0,291800.0
?,759.0,934.496706,7416.243875,0.0,5.00,10.0,10.0,89982.0


In [108]:
df[df["age"]=="?"]["km"].value_counts(dropna=False)

NaN        838
10.0       369
1.0        146
5.0         58
20.0        32
15.0        21
0.0         19
11.0        12
8.0         11
50.0        10
100.0        8
12.0         8
7.0          7
3.0          4
9.0          4
4.0          3
25.0         3
250.0        3
30.0         3
3000.0       2
22627.0      2
39962.0      2
2.0          2
19500.0      1
11000.0      1
85000.0      1
4307.0       1
89692.0      1
77.0         1
3500.0       1
68485.0      1
5000.0       1
141.0        1
150.0        1
34164.0      1
142.0        1
32084.0      1
81800.0      1
11200.0      1
20768.0      1
4500.0       1
40.0         1
784.0        1
89982.0      1
500.0        1
325.0        1
6100.0       1
196.0        1
6.0          1
60.0         1
497.0        1
99.0         1
281.0        1
Name: km, dtype: int64

In [112]:
cond1 = (df['km'] < 10000)
cond2 = ((df['km'] >= 10000) & (df['km'] < 28000))
cond3 = ((df['km'] >= 28000) & (df['km'] < 50000))
cond4 = (df['km'] >= 50000)

In [113]:
df.loc[cond1,'age'] = df.loc[cond1,'age'].replace('?', 0)
df.loc[cond2,'age'] = df.loc[cond2,'age'].replace('?', 1)
df.loc[cond3,'age'] = df.loc[cond3,'age'].replace('?', 2)
df.loc[cond4,'age'] = df.loc[cond4,'age'].replace('?', 3)

In [114]:
df.groupby('age').km.mean()Co2

age
0.0     1647.362609
1.0    18035.130067
2.0    41748.576923
3.0    77450.062789
?               NaN
Name: km, dtype: float64

In [115]:
df["age"].value_counts(dropna=False)

1.0    4528
3.0    3679
0.0    3597
2.0    3277
?       838
Name: age, dtype: int64

In [116]:
df.groupby(['make_model', 'age']).km.describe()

count          mean           std      min       25%  \
make_model     age                                                         
Audi A1        0.0  569.0   2103.458699   3258.849662      0.0     10.00   
               1.0  744.0  13806.143817   7975.692798      1.0   7466.75   
               2.0  432.0  25821.712963  18608.364225     10.0  14252.50   
               3.0  629.0  54332.286169  26281.268811   3150.0  34914.00   
               ?      0.0           NaN           NaN      NaN       NaN   
Audi A2        1.0    1.0  26166.000000           NaN  26166.0  26166.00   
Audi A3        0.0  671.0   1517.816692   6548.643220      0.0     10.00   
               1.0  776.0  18410.524485  11054.419659      1.0  11200.00   
               2.0  675.0  43853.140741  27349.704178     15.0  25000.00   
               3.0  818.0  90092.982885  36464.155900     35.0  62863.25   
               ?      0.0           NaN           NaN      NaN       NaN   
Opel Astra     0.0  541.0   2351.275416   5103.475957      0.0     10.00   
               1.0  721.0  21442.708738  10900.189850      5.0  14000.00   
               2.0  587.0  49981.778535  31163.248018     30.0  25201.50   
               3.0  521.0  89086.696737  42021.315246  10027.0  58021.00   
               ?      0.0           NaN           NaN      NaN       NaN   
Opel Corsa     0.0  647.0    913.544049   3591.393880      0.0     10.00   
               1.0  684.0  17431.599415  11739.606143      5.0   8381.25   
               2.0  386.0  39942.383420  25855.344959     99.0  21075.00   
               3.0  430.0  62947.974419  36418.877911   1800.0  34803.75   
               ?      0.0           NaN           NaN      NaN       NaN   
Opel Insignia  0.0  435.0   2419.908046   4165.213288      0.0     10.00   
               1.0  767.0  22179.207301   8486.335463      1.0  16217.50   
               2.0  589.0  47732.103565  32834.327989      1.0  22561.00   
               3.0  568.0  93182.341549  41020.813521  10791.0  63000.00   
               ?      0.0           NaN           NaN      NaN       NaN   
Renault Clio   0.0  332.0    581.716867   5865.375278      0.0      9.75   
               1.0  564.0  15396.517730  13314.804279      1.0   5296.25   
               2.0  437.0  36660.153318  21098.396486     20.0  23000.00   
               3.0  413.0  68615.435835  35580.166690     10.0  42208.00   
               ?      0.0           NaN           NaN      NaN       NaN   
Renault Duster 0.0   16.0    101.000000      0.000000    101.0    101.00   
               ?      0.0           NaN           NaN      NaN       NaN   
Renault Espace 0.0  239.0   1496.104603   3501.073586      0.0     10.00   
               1.0  233.0  14226.330472  11099.268820      1.0   5250.00   
               2.0  170.0  41886.635294  27216.915067      1.0  22313.75   
               3.0  300.0  74400.566667  34189.092815   3532.0  49975.00   
               ?      0.0           NaN           NaN      NaN       NaN   

                        50%        75%       max  
make_model     age                                
Audi A1        0.0    100.0    3333.00   18000.0  
               1.0  12413.0   20309.75   47000.0  
               2.0  20730.5   32028.75  148257.0  
               3.0  50000.0   65500.00  192000.0  
               ?        NaN        NaN       NaN  
Audi A2        1.0  26166.0   26166.00   26166.0  
Audi A3        0.0     10.0     100.00  127022.0  
               1.0  18000.0   24215.00  136000.0  
               2.0  36677.0   55251.00  158000.0  
               3.0  88000.0  112562.50  291800.0  
               ?        NaN        NaN       NaN  
Opel Astra     0.0     50.0    3698.00   88000.0  
               1.0  22000.0   28466.00   67500.0  
               2.0  44179.0   69188.00  188000.0  
               3.0  84900.0  112388.00  260000.0  
               ?        NaN        NaN       NaN  
Opel Corsa     0.0     10.0     100.00   76300.0  
          

In [117]:
df.groupby(['make_model',"body_type", 'age']).price.describe()

count          mean           std      min  \
make_model     body_type     age                                               
Audi A1        Compact       0.0  198.0  23277.434343   3510.405726  14900.0   
                             1.0  268.0  18596.041045   2659.909876  13980.0   
                             2.0  161.0  16602.807453   2085.383788  10999.0   
                             3.0  234.0  14532.910256   1908.908968   9950.0   
                             ?    178.0  23996.264045   3383.852477  16220.0   
               Coupe         2.0    1.0  15900.000000           NaN  15900.0   
                             3.0    1.0  13950.000000           NaN  13950.0   
               Other         0.0    8.0  23826.250000   2057.439029  21490.0   
                             1.0    3.0  16796.666667    178.978583  16590.0   
                             2.0    1.0  23490.000000           NaN  23490.0   
                             ?      1.0  21850.000000           NaN  21850.0   
               Sedans        0.0  368.0  24158.086957   3196.984679  15990.0   
                             1.0  465.0  18394.937634   2588.158787  13450.0   
                             2.0  266.0  16605.394737   2223.979302  11600.0   
                             3.0  389.0  14466.899743   1890.277837   8999.0   
                             ?     50.0  23336.560000   3815.326231  15980.0   
               Station wagon 0.0    2.0  28944.500000     77.074639  28890.0   
                             1.0   11.0  17410.909091   2166.030723  15750.0   
                             2.0    3.0  17613.333333   2953.816740  14750.0   
                             3.0    5.0  14358.000000   1276.891342  12950.0   
               Van           0.0    1.0  29000.000000           NaN  29000.0   
Audi A2        Off-Road      1.0    1.0  28200.000000           NaN  28200.0   
Audi A3        Compact       0.0   17.0  29345.470588   8767.064896  15500.0   
                             1.0   29.0  22030.931034   4270.468935  14500.0   
                             2.0   39.0  20489.769231   8208.444559  13790.0   
                             3.0   93.0  15303.795699   2256.722684  12900.0   
                             ?      4.0  25582.500000   1185.109137  24542.0   
               Convertible   1.0    3.0  29664.333333  23557.901357  10893.0   
                             2.0    4.0  28562.500000   5301.159464  20900.0   
                             3.0    1.0  20000.000000           NaN  20000.0   
               Coupe         2.0    1.0  19000.000000           NaN  19000.0   
                             3.0    3.0  17326.666667    583.123772  16990.0   
               Other         1.0    4.0  23422.500000   3548.054631  20200.0   
                             2.0    4.0  18675.000000   1351.850583  17650.0   
                             3.0    5.0  15970.000000   2392.070233  12550.0   
                             ?      3.0  25990.000000   1322.875656  24490.0   
               Sedans        0.0  626.0  24909.897764   2354.990202  18820.0   
                             1.0  709.0  22544.715092   5224.410267  14250.0   
                             2.0  550.0  19980.358182   3721.376402  10550.0   
                             3.0  647.0  16702.831530   2900.228429  10950.0   
                             ?     66.0  24395.242424   2372.568696  19750.0   
               Station wagon 0.0   69.0  25219.000000   5011.256638  17990.0   
                             1.0   32.0  21252.593750   3201.415069   5555.0   
                             2.0   72.0  19010.875000   1522.621645  14590.0   
                             3.0   67.0  15739.507463   1883.567363  11790.0   
                             ?     42.0  23558.666667   1687.726392  18980.0   
Opel Astra     Compact       0.0   51.0  23056.117647   7391.480059  16500.0   
                             1.0   39.0  14717.538462   1266.384531  11999.0   
                             2.0  

In [118]:
df['age'].replace('?',0, inplace=True)

In [119]:
df.groupby('age').km.mean()

age
0.0     1647.362609
1.0    18035.130067
2.0    41748.576923
3.0    77450.062789
Name: km, dtype: float64

In [120]:
df["age"].value_counts(dropna=False)

1.0    4528
0.0    4435
3.0    3679
2.0    3277
Name: age, dtype: int64

## body_type

In [125]:
df['body_type'].value_counts(dropna=False)

Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
NaN                60
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [127]:
df.body_type.replace("Other", np.nan, inplace=True) 

In [128]:
df['body_type'].value_counts(dropna=False)

Sedans           7903
Station wagon    3553
Compact          3153
Van               783
NaN               350
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [129]:
df["body_type"].mode()

0    Sedans
dtype: object

In [130]:
df["body_type"].mode()[0]

'Sedans'

In [131]:
for group in list(df["make_model"].unique()):
    cond = df["make_model"]==group
    mode = list(df[cond]["body_type"].mode())
    if mode != []:
        df.loc[cond, "body_type"] = df.loc[cond, "body_type"].fillna(df[cond]["body_type"].mode()[0])
    else:
        df.loc[cond, "body_type"] = df.loc[cond, "body_type"].fillna(df["body_type"].mode()[0])

In [132]:
df['body_type'].value_counts(dropna=False)

Sedans           8005
Station wagon    3678
Compact          3242
Van               817
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [133]:
def fill_most(df, group_col, col_name):
    '''Fills the missing values with the most existing value (mode) in the relevant column according to single-stage grouping'''
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        mode = list(df[cond][col_name].mode())
        if mode != []:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[cond][col_name].mode()[0])
        else:
            df.loc[cond, col_name] = df.loc[cond, col_name].fillna(df[col_name].mode()[0])
    print("Number of NaN : ",df[col_name].isnull().sum())
    print("------------------")
    print(df[col_name].value_counts(dropna=False))

In [134]:
fill_most(df, "make_model", "body_type")

Number of NaN :  0
------------------
Sedans           8005
Station wagon    3678
Compact          3242
Van               817
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64


## Price 

In [30]:
df['price'].value_counts(dropna=False)

14990    154
15990    151
10990    139
15900    106
17990    102
        ... 
31940      1
15540      1
21649      1
27790      1
34815      1
Name: price, Length: 2956, dtype: int64

In [31]:
df.price.isnull().sum()

0

In [32]:
df.price.describe()

count    15919.000000
mean     18019.896727
std       7386.169409
min         13.000000
25%      12850.000000
50%      16900.000000
75%      21900.000000
max      74600.000000
Name: price, dtype: float64

## Vat 

In [33]:
df.vat= df.vat.fillna(method='ffill').fillna(method='bfill')

In [34]:
df.vat.value_counts(dropna=False)

VAT deductible      15048
Price negotiable      871
Name: vat, dtype: int64

## Km 

In [121]:
df.groupby("age").km.mean()

age
0.0     1647.362609
1.0    18035.130067
2.0    41748.576923
3.0    77450.062789
Name: km, dtype: float64

In [122]:
df.groupby("age").km.transform("mean")

0        77450.062789
1        41748.576923
2        77450.062789
3        77450.062789
4        77450.062789
             ...     
15914     1647.362609
15915     1647.362609
15916     1647.362609
15917     1647.362609
15918     1647.362609
Name: km, Length: 15919, dtype: float64

In [123]:
df["km"].fillna(df.groupby("age").km.transform("mean"), inplace=True)

In [124]:
df.km.value_counts(dropna=False)

10.000000        1045
1647.362609       985
1.000000          367
5.000000          170
50.000000         148
                 ... 
160542.000000       1
20719.000000        1
91910.000000        1
39860.000000        1
57889.000000        1
Name: km, Length: 6692, dtype: int64

# Previous Owners

In [135]:
df['Previous_Owners'].value_counts(dropna=False) # NaN    6640 adet var

1.0    8294
NaN    6640
2.0     778
0.0     188
3.0      17
4.0       2
Name: Previous_Owners, dtype: int64

In [136]:
df["Previous_Owners"].fillna("?", inplace = True)

In [137]:
df["Previous_Owners"].value_counts(dropna=False)

1.0    8294
?      6640
2.0     778
0.0     188
3.0      17
4.0       2
Name: Previous_Owners, dtype: int64

In [138]:
df.groupby(['make_model', 'age', 'Previous_Owners']).km.describe()

count           mean           std  \
make_model     age Previous_Owners                                       
Audi A1        0.0 0.0               45.0     958.365063    815.361604   
                   1.0              239.0    3069.542783   3460.401760   
                   2.0                1.0    3000.000000           NaN   
                   ?                521.0    1549.991910   2303.799717   
               1.0 0.0                1.0   15000.000000           NaN   
                   1.0              496.0   13772.192319   8099.460976   
                   2.0               14.0   13734.285714  10613.039001   
                   ?                236.0   13930.461864   7544.057200   
               2.0 0.0                1.0   68002.000000           NaN   
                   1.0              195.0   23858.369231  19666.375578   
                   2.0               74.0   20918.945946  14313.355953   
                   3.0                1.0   18890.000000           NaN   
                   4.0                1.0   19700.000000           NaN   
                   ?                160.0   30300.025000  18023.499267   
               3.0 0.0                4.0   58176.750000   5569.646510   
                   1.0              401.0   54041.192020  27128.753962   
                   2.0               82.0   49948.817073  18268.135159   
                   3.0                1.0   45000.000000           NaN   
                   ?                141.0   57666.524823  27893.546330   
Audi A2        1.0 1.0                1.0   26166.000000           NaN   
Audi A3        0.0 0.0               49.0     880.641384    823.805038   
                   1.0              257.0    1953.461922   3715.728211   
                   ?                521.0    1401.636133   6946.583315   
               1.0 0.0                5.0   28967.000000   8648.989189   
                   1.0              380.0   17260.626658  10710.878608   
                   2.0               11.0   17966.818182   6209.465191   
                   ?                381.0   19430.692913  11365.016061   
               2.0 1.0              351.0   47381.068376  30979.305235   
                   2.0               47.0   28973.489362  19029.481561   
                   ?                277.0   41907.440433  22179.629617   
               3.0 0.0                1.0   64500.000000           NaN   
                   1.0              458.0   94950.720524  38584.467935   
                   2.0               53.0   67559.188679  27702.964742   
                   3.0                2.0   71800.000000  62508.239457   
                   ?                304.0   86907.542763  32497.991118   
Opel Astra     0.0 0.0               12.0     555.870870    806.229506   
                   1.0              253.0    3601.620900   6588.867489   
                   2.0                1.0    7700.000000           NaN   
                   ?                425.0    1396.620831   2349.956871   
               1.0 0.0                1.0   30800.000000           NaN   
                   1.0              531.0   21259.041431   9582.280779   
                   2.0               24.0   19963.458333   9864.029510   
                   ?                171.0   22046.372985  14244.299789   
               2.0 1.0              354.0   52083.943503  34251.398310   
                   2.0               54.0   35318.259259  22641.366194   
                   3.0                1.0   62000.000000           NaN   
                   ?                178.0   50182.033708  25430.913374   
               3.0 1.0              349.0   95006.730659  41188.742541   
                   2.0               55.0   69682.436364  41233.912306   
                   3.0                2.0   60099.500000   5090.461718   
                   ?                115.0   80905.104348  41216.973714   
Opel Corsa     0.0 0.0               12.0     688.567754    846.373934   
                   1.0              443.

In [139]:
df[(df["make_model"]=="Renault Duster") & (df["Previous_Owners"] == "?")]["km"]

14894    1647.362609
14895    1647.362609
14896     101.000000
14897    1647.362609
14898     101.000000
14899     101.000000
14900    1647.362609
14901     101.000000
14903     101.000000
14904    1647.362609
14905    1647.362609
14906     101.000000
14907     101.000000
14908    1647.362609
14909    1647.362609
14910     101.000000
14911    1647.362609
14912     101.000000
14913    1647.362609
14914     101.000000
14915     101.000000
14916     101.000000
14917    1647.362609
14918    1647.362609
14919    1647.362609
14920     101.000000
14921    1647.362609
14922    1647.362609
14923     101.000000
14924    1647.362609
14925     101.000000
14926     101.000000
14927    1647.362609
Name: km, dtype: float64

In [145]:
cond = (df["make_model"]=="Renault Duster") & (df["Previous_Owners"] == "?")
df.loc[cond, "Previous_Owners"] = df.loc[cond, "Previous_Owners"].replace("?", 0.0)

In [146]:
df["Previous_Owners"].value_counts(dropna=False)

1.0    8294
NaN    6640
2.0     778
0.0     188
3.0      17
4.0       2
Name: Previous_Owners, dtype: int64

In [147]:
df["Previous_Owners"].replace("?", np.nan, inplace=True)

In [148]:
for group in list(df["age"].unique()):
    cond = df["age"]==group
    df.loc[cond, "Previous_Owners"] = df.loc[cond, "Previous_Owners"].fillna(method="ffill").fillna(method="bfill")
df["Previous_Owners"] = df["Previous_Owners"].fillna(method="ffill").fillna(method="bfill")

In [48]:
df["Previous_Owners"].value_counts(dropna=False)

1.0    14162
2.0     1172
0.0      554
3.0       29
4.0        2
Name: Previous_Owners, dtype: int64

In [154]:
def fill_prop(df, group_col, col_name):
    '''Fills the missing values with "ffill and bfill method" according to single-stage grouping'''
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        df.loc[cond, col_name].df['body_type'].value_counts(dropna=False) = df.loc[cond, col_name].fillna(method="ffill").fillna(method="bfill")
    df[col_name] = df[col_name].fillna(method="ffill").fillna(method="bfill")
   

SyntaxError: cannot assign to function call (<ipython-input-154-63dde0c08b76>, line 5)

In [149]:
fill_prop(df, "age", "Previous_Owners")

# hp_kW

In [49]:
df['hp_kW'].value_counts(dropna=False)

85.0     2542
66.0     2122
81.0     1402
100.0    1308
110.0    1112
70.0      888
125.0     707
51.0      695
55.0      569
118.0     516
92.0      466
121.0     392
147.0     380
77.0      345
56.0      286
54.0      276
103.0     253
87.0      232
165.0     194
88.0      177
60.0      160
162.0      98
NaN        88
74.0       81
96.0       72
71.0       59
101.0      47
67.0       40
154.0      39
122.0      35
119.0      30
164.0      27
135.0      24
82.0       22
52.0       22
1.0        20
78.0       20
146.0      18
294.0      18
141.0      16
57.0       10
120.0       8
104.0       8
191.0       7
112.0       7
155.0       6
117.0       6
184.0       5
90.0        4
76.0        4
65.0        4
149.0       3
98.0        3
93.0        3
80.0        3
168.0       3
150.0       2
63.0        2
140.0       2
86.0        2
89.0        2
40.0        2
167.0       2
53.0        2
228.0       2
127.0       2
143.0       2
270.0       2
9.0         1
44.0        1
123.0       1
195.0 

In [50]:
df["hp_kW"].fillna("?", inplace=True)

In [51]:
df.groupby(["make_model", "body_type","hp_kW"]).price.describe()

count          mean           std  \
make_model     body_type     hp_kW                                       
Audi A1        Compact       60.0     58.0  15189.827586   1663.611419   
                             66.0    162.0  15398.209877   1934.300783   
                             70.0    332.0  17983.262048   3315.814057   
                             71.0     32.0  20831.593750   2410.839780   
                             85.0    330.0  22604.103030   4572.568357   
                             86.0      1.0  14295.000000           NaN   
                             92.0     90.0  18029.055556   3027.431352   
                             93.0      2.0  21447.500000    774.281925   
                             110.0    20.0  23620.450000   3777.343838   
                             141.0     2.0  22495.000000    841.457070   
                             147.0     8.0  28797.125000    355.355933   
                             ?         2.0  19125.000000   5126.524164   
               Coupe         70.0      1.0  13950.000000           NaN   
                             85.0      1.0  15900.000000           NaN   
               Sedans        60.0     95.0  15218.168421   1624.266962   
                             66.0    296.0  15922.797297   2331.607661   
                             67.0     12.0  15432.416667   2778.190564   
                             70.0    455.0  17282.068132   3210.456485   
                             71.0     24.0  19177.250000   2539.495195   
                             85.0    555.0  21609.522523   4822.007178   
                             87.0      4.0  14865.000000    750.000000   
                             92.0     79.0  19078.974684   2790.208934   
                             93.0      1.0  17900.000000           NaN   
                             110.0     9.0  22591.333333   4188.634533   
                             112.0     1.0  18399.000000           NaN   
                             140.0     1.0  23490.000000           NaN   
                             141.0    14.0  21232.000000   2597.465895   
                             147.0     5.0  30729.600000   4008.781710   
               Station wagon 60.0      3.0  15106.666667   1890.881629   
                             66.0      7.0  16018.428571   1779.762330   
                             70.0      3.0  17420.000000   3283.473770   
                             85.0      4.0  22702.750000   7207.559799   
                             92.0      3.0  17623.000000   3738.860388   
                             110.0     1.0  20650.000000           NaN   
               Van           147.0     1.0  29000.000000           NaN   
Audi A2        Off-Road      85.0      1.0  28200.000000           NaN   
Audi A3        Compact       81.0     50.0  17147.920000   2283.775292   
                             82.0     11.0  17042.090909   1353.480658   
                             85.0     53.0  22656.716981   6328.250981   
                             87.0      3.0  22195.000000    349.887125   
                             92.0     43.0  13824.883721    984.707769   
                             110.0    15.0  25249.333333   7908.351008   
                             112.0     1.0  17894.000000           NaN   
                             135.0     3.0  15950.000000      0.000000   
                             150.0     1.0  24950.000000           NaN   
                             294.0     1.0  67600.000000           NaN   
                             ?         1.0  18700.000000           NaN   
               Convertible   81.0      1.0  20900.000000           NaN   
                             110.0     5.0  24848.600000   9248.720712   
                             135.0     1.0  22000.000000           NaN   
                             228.0     1.0  56100.000000           NaN   
               Coupe         81.0      3.0  17326.666667    583.123772   
                             85.0      1.

In [52]:
df["hp_kW"].replace("?", np.nan, inplace=True)

In [53]:
fill(df, "make_model", "body_type", "hp_kW", "mode")

In [54]:
df['hp_kW'].isnull().value_counts(dropna=False)

False    15919
Name: hp_kW, dtype: int64

## Type -1 tane NaN

In [55]:
df["Type"].value_counts(dropna=False)

Used              11096
New                1650
Pre-registered     1364
Employee's car     1011
Demonstration       796
NaN                   2
Name: Type, dtype: int64

In [56]:
df.Type.fillna("?", inplace=True)

In [57]:
df['Type'].value_counts(dropna=False)

Used              11096
New                1650
Pre-registered     1364
Employee's car     1011
Demonstration       796
?                     2
Name: Type, dtype: int64

In [58]:
df.groupby(["Type", "make_model", "age"]).km.describe()

count           mean           std  \
Type           make_model     age                                       
?              Audi A3        0.0    1.0    1647.362609           NaN   
                              3.0    1.0  115137.000000           NaN   
Demonstration  Audi A1        0.0  110.0    3784.227273  2.498415e+03   
                              1.0   19.0    9931.157895  5.209543e+03   
                              2.0    6.0   22963.500000  1.780268e+04   
               Audi A3        0.0   41.0    4971.878049  3.092353e+03   
                              1.0   16.0   12801.187500  1.092015e+04   
                              2.0    5.0   27950.000000  9.145354e+03   
               Opel Astra     0.0  148.0    2893.375852  2.879037e+03   
                              1.0   16.0    7319.937500  7.499404e+03   
                              2.0    1.0   68500.000000           NaN   
               Opel Corsa     0.0   75.0    1248.990180  1.687252e+03   
                              1.0   14.0    2915.500000  5.334489e+03   
               Opel Insignia  0.0  161.0    3936.657769  3.983602e+03   
                              1.0    6.0   13721.000000  1.123745e+04   
                              2.0    4.0   27175.000000  4.251568e+03   
                              3.0    1.0   56000.000000           NaN   
               Renault Clio   0.0   39.0     640.309944  1.211087e+03   
                              1.0   33.0    5555.848485  6.384845e+03   
                              2.0    3.0   10014.000000  9.124492e+03   
                              3.0    1.0   35000.000000           NaN   
               Renault Espace 0.0   61.0    4413.099800  5.077255e+03   
                              1.0   28.0   10118.928571  7.526320e+03   
                              2.0    8.0   24883.000000  2.253202e+04   
Employee's car Audi A1        0.0   53.0    4506.610615  4.376534e+03   
                              1.0  131.0   12199.404580  7.558408e+03   
                              2.0    8.0   17377.750000  5.795687e+03   
                              3.0    5.0   48103.800000  3.032771e+04   
               Audi A2        1.0    1.0   26166.000000           NaN   
               Audi A3        0.0   31.0    3869.451613  5.402688e+03   
                              1.0  157.0   18197.566879  8.303327e+03   
                              2.0   32.0   31584.750000  1.494953e+04   
                              3.0    8.0   56954.625000  2.679657e+04   
               Opel Astra     0.0   18.0    5451.055556  2.971350e+03   
                              1.0  121.0   23088.033058  8.072127e+03   
                              2.0   12.0   52783.000000  2.095452e+04   
                              3.0    1.0   95000.000000           NaN   
               Opel Corsa     0.0   13.0    3255.566355  5.436737e+03   
                              1.0  115.0   16942.113043  8.966013e+03   
                              2.0   20.0   42133.550000  1.076379e+04   
                              3.0    1.0  103000.000000           NaN   
               Opel Insignia  0.0    6.0    7423.727101  1.475477e+04   
                              1.0  134.0   23911.074627  6.252446e+03   
                              2.0    9.0   48409.000000  4.510808e+04   
               Renault Clio   0.0    5.0    1459.145043  1.349267e+03   
                              1.0   79.0   24639.520634  1.062313e+04   
                              2.0   12.0   46621.083333  2.206475e+04   
                              3.0    1.0   60892.000000           NaN   
               Renault Espace 0.0    4.0    2448.681304  2.428542e+03   
                              1.0   27.0   15418.925926  1.068013e+04   
                              2.0    6.0   22298.166667  7.540583e+03   
                              3.0    1.0   50054.000000           NaN   
New            Audi A1        0.0  328.0    1154.612919  7.507992e+02   
               Audi 

In [59]:
#cond1 = (df['make_model'] == "Audi A3") & (df["age"] == 0)
cond2 = (df['make_model'] == "Audi A3") & (df["age"] == 3)

In [60]:
#df.loc[cond1,'Type'] = df.loc[cond1,'Type'].replace('?','New')
df.loc[cond2,'Type'] = df.loc[cond2,'Type'].replace('?','Used')

In [61]:
df['Type'].value_counts(dropna=False)

Used              11097
New                1650
Pre-registered     1364
Employee's car     1011
Demonstration       796
?                     1
Name: Type, dtype: int64

In [62]:
df.Type.isnull().sum()

0

## Inspection_new

In [63]:
df["Inspection_new"].value_counts(dropna=False)

NaN    11987
Yes     3932
Name: Inspection_new, dtype: int64

In [64]:
df["Inspection_new"].fillna("?", inplace=True)

In [65]:
df["Inspection_new"].value_counts(dropna=False)

?      11987
Yes     3932
Name: Inspection_new, dtype: int64

In [66]:
df.groupby(["make_model", "body_type", "age", "Inspection_new"]).price.describe()

count          mean  \
make_model     body_type     age Inspection_new                        
Audi A1        Compact       0.0 ?               243.0  24206.370370   
                                 Yes             133.0  22542.248120   
                             1.0 ?               150.0  19065.306667   
                                 Yes             118.0  17999.516949   
                             2.0 ?               115.0  16425.669565   
                                 Yes              46.0  17045.652174   
                             3.0 ?               128.0  14745.820312   
                                 Yes             106.0  14275.811321   
               Coupe         2.0 ?                 1.0  15900.000000   
                             3.0 ?                 1.0  13950.000000   
               Sedans        0.0 ?               362.0  24130.350829   
                                 Yes              65.0  23604.261538   
                             1.0 ?               323.0  18637.198142   
                                 Yes             145.0  17822.213793   
                             2.0 ?               216.0  16726.495370   
                                 Yes              51.0  16227.490196   
                             3.0 ?               276.0  14393.514493   
                                 Yes             113.0  14646.141593   
               Station wagon 0.0 ?                 2.0  28944.500000   
                             1.0 ?                 6.0  18023.333333   
                                 Yes               5.0  16676.000000   
                             2.0 ?                 1.0  17440.000000   
                                 Yes               2.0  17700.000000   
                             3.0 ?                 1.0  13999.000000   
                                 Yes               4.0  14447.750000   
               Van           0.0 ?                 1.0  29000.000000   
Audi A2        Off-Road      1.0 ?                 1.0  28200.000000   
Audi A3        Compact       0.0 ?                20.0  28815.150000   
                                 Yes               1.0  24900.000000   
                             1.0 ?                29.0  22030.931034   
                             2.0 ?                39.0  20489.769231   
                             3.0 ?                90.0  15260.144444   
                                 Yes               3.0  16613.333333   
               Convertible   1.0 ?                 3.0  29664.333333   
                             2.0 ?                 4.0  28562.500000   
                             3.0 ?                 1.0  20000.000000   
               Coupe         2.0 ?                 1.0  19000.000000   
                             3.0 ?                 3.0  17326.666667   
               Sedans        0.0 ?               543.0  24980.578269   
                                 Yes             152.0  24455.250000   
                             1.0 ?               635.0  22679.171654   
                                 Yes              78.0  21495.115385   
                             2.0 ?               467.0  20109.558887   
                                 Yes              92.0  19205.467391   
                             3.0 ?               537.0  16834.685289   
                                 Yes             117.0  16062.017094   
               Station wagon 0.0 ?                85.0  24797.305882   
                                 Yes              26.0  23915.538462   
                             1.0 ?                26.0  21163.961538   
                                 Yes               6.0  21636.666667   
                             2.0 ?                62.0  18927.951613   
                                 Yes              10.0  19525.000000   
                             3.0 ?                33.0  15931.787879   
                                 Yes              34.0  15552.882353   
Opel Astra     Co

In [67]:
df["Inspection_new"].replace("?", "No", inplace=True)

In [68]:
df["Inspection_new"].value_counts(dropna=False)

No     11987
Yes     3932
Name: Inspection_new, dtype: int64

In [69]:
df["Inspection_new"].replace(["Yes", "No"], [1,0], inplace = True)

In [70]:
df["Inspection_new"].value_counts(dropna=False)

0    11987
1     3932
Name: Inspection_new, dtype: int64

In [71]:
df['Inspection_new'].value_counts(normalize=True, dropna=False)

0    0.753
1    0.247
Name: Inspection_new, dtype: float64

## Warranty 

In [72]:
df.Warranty.value_counts(dropna=False)


NaN     11066
12.0     2594
24.0     1118
60.0      401
36.0      279
48.0      149
6.0       125
72.0       59
3.0        33
23.0       11
18.0       10
20.0        7
25.0        6
2.0         5
50.0        4
26.0        4
16.0        4
19.0        3
1.0         3
4.0         3
13.0        3
34.0        3
45.0        2
14.0        2
17.0        2
11.0        2
46.0        2
28.0        2
21.0        2
22.0        2
9.0         2
30.0        1
33.0        1
56.0        1
40.0        1
7.0         1
15.0        1
8.0         1
10.0        1
49.0        1
47.0        1
65.0        1
Name: Warranty, dtype: int64

In [73]:
df["Warranty"].fillna("?", inplace = True)

In [74]:
df.groupby(['make_model', 'age', 'Warranty']).price.describe()

count          mean          std      min  \
make_model     age Warranty                                              
Audi A1        0.0 12.0       38.0  23846.342105  2978.849554  17900.0   
                   18.0        1.0  19995.000000          NaN  19995.0   
                   24.0      107.0  23766.308411  3338.105537  14900.0   
                   36.0       26.0  25514.423077  3205.835906  19900.0   
                   48.0       18.0  24441.666667  2516.781909  19650.0   
                   56.0        1.0  21760.000000          NaN  21760.0   
                   60.0       14.0  21829.857143   717.879447  20990.0   
                   ?         601.0  23854.851913  3442.694702  15980.0   
               1.0 6.0         9.0  22166.666667   743.303437  20900.0   
                   12.0       64.0  18106.406250  2408.814308  14220.0   
                   13.0        1.0  19840.000000          NaN  19840.0   
                   20.0        1.0  17950.000000          NaN  17950.0   
                   23.0        1.0  20950.000000          NaN  20950.0   
                   24.0       88.0  18378.454545  2614.704431  14500.0   
                   26.0        1.0  15840.000000          NaN  15840.0   
                   36.0       26.0  18011.692308  2314.021725  15500.0   
                   46.0        1.0  15980.000000          NaN  15980.0   
                   47.0        1.0  16370.000000          NaN  16370.0   
                   48.0       26.0  17903.076923  2068.171694  14900.0   
                   49.0        1.0  16270.000000          NaN  16270.0   
                   50.0        3.0  17903.333333  1342.882472  16370.0   
                   60.0       11.0  16426.000000   434.339729  15900.0   
                   ?         513.0  18542.497076  2664.306817  13450.0   
               2.0 3.0         2.0  20740.000000  3889.087297  17990.0   
                   6.0         3.0  18443.333333  2475.284495  15990.0   
                   12.0       80.0  16187.125000  1918.722659  13500.0   
                   24.0       33.0  16858.454545  2723.500403  11990.0   
                   36.0        8.0  17129.250000  2677.734690  14400.0   
                   48.0        4.0  19800.000000   115.470054  19700.0   
                   60.0        1.0  14990.000000          NaN  14990.0   
                   ?         301.0  16621.209302  2131.490525  10999.0   
               3.0 3.0         2.0  13235.000000   346.482323  12990.0   
                   6.0         9.0  15632.000000  2329.396059  12980.0   
                   12.0      120.0  14324.950000  1914.102194   9950.0   
                   23.0        1.0  12170.000000          NaN  12170.0   
                   24.0       13.0  14540.000000  1970.786645  12500.0   
                   60.0        4.0  13467.250000   391.156725  12999.0   
                   ?         480.0  14526.779167  1877.215643   8999.0   
Audi A2        1.0 ?           1.0  28200.000000          NaN  28200.0   
Audi A3        0.0 3.0         1.0  17990.000000          NaN  17990.0   
                   11.0        1.0  26795.000000          NaN  26795.0   
                   12.0       46.0  24362.782609  1527.389391  21900.0   
                   19.0        2.0  26273.000000   388.908730  25998.0   
                   24.0      104.0  25156.144231  4333.165758  20290.0   
                   36.0       73.0  25524.780822  1657.660058  20000.0   
                   48.0       19.0  24593.684211  1326.973208  21900.0   
                   60.0        4.0  23942.500000  2531.736887  20990.0   
                   72.0        1.0  23990.000000          NaN  23990.0   
                   ?         576.0  24874.692708  2930.115451  15500.0   
               1.0 3.0         1.0  28490.000000          NaN  28490.0   
                   12.0      121.0  21765.223140  3092.652116  14500.0   
                   16.0        1.0  10893.000000          NaN  10893.0   
                   18.0        2.

In [75]:
df.drop("Warranty", axis=1, inplace=True)

# Body Color ? CLEANING'DEN DAHİL EDİLİP TEKRAR ANALİZ YAPILACAK

In [76]:
df["Body Color"]

0        ['\n', 'Black', '\n']
1          ['\n', 'Red', '\n']
2        ['\n', 'Black', '\n']
3        ['\n', 'Brown', '\n']
4        ['\n', 'Black', '\n']
                 ...          
15914     ['\n', 'Grey', '\n']
15915     ['\n', 'Grey', '\n']
15916    ['\n', 'White', '\n']
15917     ['\n', 'Grey', '\n']
15918     ['\n', 'Grey', '\n']
Name: Body Color, Length: 15919, dtype: object

In [77]:
df['Body Color'].str[1].str.strip().value_counts(dropna=False)

'      15322
NaN      597
Name: Body Color, dtype: int64

In [78]:
df['Body Color'].map(type).value_counts()

<class 'str'>      15322
<class 'float'>      597
Name: Body Color, dtype: int64

In [79]:
df["Body Color"].isnull().sum()

597

In [80]:
df['Body Color'].str[1].value_counts(normalize=True, dropna=False).to_frame()

,Body Color
',0.962498
NaN,0.037502


# Paint Type ? CLEANING'DEN DAHİL EDİLİP TEKRAR ANALİZ YAPILACAK

In [81]:
df["Paint Type"]

0        \nMetallic\n
1                 NaN
2        \nMetallic\n
3        \nMetallic\n
4        \nMetallic\n
             ...     
15914    \nMetallic\n
15915    \nMetallic\n
15916             NaN
15917             NaN
15918    \nMetallic\n
Name: Paint Type, Length: 15919, dtype: object

In [82]:
df['Paint Type'] = df['Paint Type'].apply(lambda item:item[0] if type(item)  == list else item)

In [83]:
df['Paint Type'].str.strip().value_counts(dropna=False)

Metallic       9794
NaN            5772
Uni/basic       347
Perl effect       6
Name: Paint Type, dtype: int64

In [84]:
df["Paint Type"].isnull().sum()/len(df["Paint Type"])*100

36.25855895470821

In [85]:
df["Paint Type"].isnull().sum()

5772

# Upholstery_type

In [86]:
df["Upholstery_type"].value_counts(dropna=False)

Cloth           8423
NaN             4871
Part leather    1499
Full leather    1009
Velour            60
alcantara         57
Name: Upholstery_type, dtype: int64

In [87]:
df["Upholstery_type"].replace(["Velour", "alcantara", "Part leather", "Full leather"], ["Cloth", "Part/Full Leather", "Part/Full Leather", "Part/Full Leather"], inplace=True)

In [88]:
df["Upholstery_type"].value_counts(dropna=False)

Cloth                8483
NaN                  4871
Part/Full Leather    2565
Name: Upholstery_type, dtype: int64

In [89]:
fill(df, "make_model", "body_type", "Upholstery_type", "ffill")

In [90]:
df["Upholstery_type"].value_counts(dropna=False)

Cloth                12238
Part/Full Leather     3681
Name: Upholstery_type, dtype: int64

# Upholstery_color

In [91]:
df["Upholstery_color"].value_counts(dropna=False)

Black     8201
NaN       5078
Grey      1376
Other      960
Brown      207
Beige       54
Blue        16
White       13
Red          9
Yellow       4
Orange       1
Name: Upholstery_color, dtype: int64

In [92]:
df.drop("Upholstery_color", axis=1, inplace=True)

# Nr. of Doors-1 NaN

In [93]:
df["Nr_of_Doors"].value_counts(dropna=False)

KeyError: 'Nr_of_Doors'

In [ ]:
fill(df, "make_model", "body_type", "Nr_of_Doors", "mode")

In [ ]:
df["Nr_of_Doors"].value_counts(dropna=False)

5.0    11786
4.0     3079
3.0      832
2.0      219
NaN        1
7.0        1
1.0        1
Name: Nr_of_Doors, dtype: int64

# Nr. of Seats-47 NaN

In [ ]:
df["Nr_of_Seats"].value_counts(dropna=False)

5.0    13336
4.0     1125
NaN      977
7.0      362
2.0      116
6.0        2
3.0        1
Name: Nr_of_Seats, dtype: int64

In [ ]:
fill(df, "make_model", "body_type", "Nr_of_Seats", "mode")

In [ ]:
df["Nr_of_Seats"].isnull().sum()

47

# Displacement_cc- 23NaN

In [ ]:
df["Displacement_cc"].value_counts(dropna=False)

1598.0     4761
999.0      2438
1398.0     1314
1399.0      749
1229.0      677
1956.0      670
1461.0      595
1490.0      559
NaN         496
1422.0      467
1197.0      353
898.0       351
1395.0      320
1968.0      301
1149.0      288
1618.0      212
1798.0      210
1498.0      196
1600.0      130
1248.0      110
1997.0      103
1364.0      102
1400.0       90
998.0        72
1500.0       50
2000.0       46
1000.0       40
1.0          36
1998.0       25
2480.0       20
1984.0       18
1200.0       18
899.0        11
1397.0       11
160.0         6
1499.0        5
929.0         5
139.0         4
900.0         4
1596.0        4
997.0         4
1199.0        3
1599.0        3
1396.0        3
1495.0        2
1300.0        2
1589.0        2
2.0           2
995.0         2
1568.0        1
1584.0        1
16000.0       1
1369.0        1
996.0         1
1686.0        1
1100.0        1
1496.0        1
1195.0        1
1896.0        1
1239.0        1
890.0         1
973.0         1
1198.0  

In [ ]:
df["Displacement_cc"].fillna("?", inplace=True)

In [ ]:
df.groupby(["make_model", "body_type","Displacement_cc"]).price.describe()

count          mean  \
make_model     body_type     Displacement_cc                         
Audi A1        Compact       929.0               5.0  16364.000000   
                             995.0               2.0  21990.000000   
                             999.0             663.0  20294.101056   
                             1000.0              7.0  19498.428571   
                             1395.0             98.0  18246.622449   
                             1422.0            158.0  15396.424051   
                             1498.0             14.0  24982.142857   
                             1596.0              1.0  15585.000000   
                             1598.0             68.0  16568.308824   
                             1600.0              1.0  16800.000000   
                             1798.0              2.0  22495.000000   
                             1984.0              8.0  28797.125000   
                             ?                  12.0  17960.833333   
               Coupe         999.0               1.0  13950.000000   
                             1598.0              1.0  15900.000000   
               Other         999.0              10.0  22905.000000   
                             1598.0              2.0  16900.000000   
                             ?                   1.0  23490.000000   
               Sedans        998.0               2.0  20990.000000   
                             999.0             872.0  19848.371560   
                             1000.0              7.0  26414.285714   
                             1197.0              1.0  15980.000000   
                             1395.0             85.0  18973.647059   
                             1422.0            302.0  15917.009934   
                             1498.0              2.0  28950.000000   
                             1596.0              1.0  17700.000000   
                             1598.0            231.0  16801.757576   
                             1600.0              1.0  13290.000000   
                             1798.0             14.0  21232.000000   
                             1984.0              5.0  30729.600000   
                             ?                  15.0  17598.666667   
               Station wagon 999.0               7.0  18067.142857   
                             1395.0              4.0  18379.750000   
                             1422.0              7.0  16018.428571   
                             1598.0              2.0  16461.000000   
                             ?                   1.0  28999.000000   
               Van           1984.0              1.0  29000.000000   
Audi A2        Off-Road      1598.0              1.0  28200.000000   
Audi A3        Compact       999.0              16.0  27854.812500   
                             1197.0              1.0  18495.000000   
                             1395.0             48.0  14657.583333   
                             1400.0              1.0  24950.000000   
                             1495.0              2.0  27611.000000   
                             1498.0              1.0  24490.000000   
                             1499.0              2.0  39995.000000   
                             1598.0             97.0  18544.206186   
                             1600.0              1.0  17900.000000   
                             1968.0             10.0  19208.800000   
                             2480.0              1.0  67600.000000   
                             ?                   2.0  16845.000000   
               Convertible   1395.0              1.0  32950.000000   
                             1498.0              1.0  10893.000000   
                             1598.0              1.0  20900.000000   
                             1968.0              4.0  25600.000000   
                             1984.0              1.0  56100.000000   
               Coupe         999.0               1.0  190

In [ ]:
df["Displacement_cc"].value_counts(dropna=False)

1598.0     4761
999.0      2438
1398.0     1314
1399.0      749
1229.0      677
1956.0      670
1461.0      595
1490.0      559
?           496
1422.0      467
1197.0      353
898.0       351
1395.0      320
1968.0      301
1149.0      288
1618.0      212
1798.0      210
1498.0      196
1600.0      130
1248.0      110
1997.0      103
1364.0      102
1400.0       90
998.0        72
1500.0       50
2000.0       46
1000.0       40
1.0          36
1998.0       25
2480.0       20
1200.0       18
1984.0       18
1397.0       11
899.0        11
160.0         6
929.0         5
1499.0        5
1596.0        4
900.0         4
997.0         4
139.0         4
1599.0        3
1199.0        3
1396.0        3
1589.0        2
1495.0        2
995.0         2
1300.0        2
2.0           2
122.0         1
1239.0        1
1896.0        1
1995.0        1
1856.0        1
1800.0        1
140.0         1
1696.0        1
1686.0        1
54.0          1
1195.0        1
16000.0       1
1350.0        1
1580.0  

In [ ]:
df["Displacement_cc"].replace("?", np.nan, inplace=True)

In [ ]:
fill(df, "make_model", "body_type", "Displacement_cc", "mode")

In [ ]:
df["Displacement_cc"].value_counts(dropna=False)

1598.0     5025
999.0      2467
1398.0     1386
1399.0      768
1229.0      678
1956.0      670
1461.0      664
1490.0      559
1422.0      467
1197.0      372
898.0       351
1395.0      320
1968.0      301
1149.0      288
1618.0      212
1798.0      210
1498.0      196
1600.0      130
1248.0      110
1997.0      103
1364.0      102
1400.0       90
998.0        72
1500.0       50
2000.0       46
1000.0       40
1.0          36
1998.0       25
NaN          23
2480.0       20
1200.0       18
1984.0       18
899.0        11
1397.0       11
160.0         6
1499.0        5
929.0         5
139.0         4
1596.0        4
997.0         4
900.0         4
1396.0        3
1199.0        3
1599.0        3
1300.0        2
1495.0        2
1589.0        2
995.0         2
2.0           2
16000.0       1
1496.0        1
1368.0        1
996.0         1
890.0         1
1584.0        1
1686.0        1
1369.0        1
1568.0        1
1239.0        1
1896.0        1
973.0         1
1100.0        1
1580.0  

# Cylinders

In [ ]:
fill(df, "make_model", "body_type", "Cylinders", "mode")

In [ ]:
df.drop("Cylinders", axis = 1, inplace = True)

# Weight- 52NaN

In [ ]:
df["Weight_kg"].value_counts(dropna=False)

NaN       6974
1163.0     574
1360.0     356
1165.0     301
1335.0     242
1135.0     213
1199.0     205
1734.0     170
1180.0     168
1503.0     165
1350.0     155
1355.0     135
1260.0     127
1275.0     112
1278.0     110
1425.0     109
1487.0     109
1255.0     108
1200.0     107
1522.0     103
1273.0     103
1280.0     102
1403.0      91
1120.0      90
1195.0      89
1659.0      89
1701.0      87
1250.0      84
1441.0      82
1308.0      80
1285.0      80
1110.0      75
1613.0      75
1279.0      72
1364.0      70
1345.0      67
1733.0      65
1685.0      64
1071.0      64
1325.0      64
1141.0      64
1230.0      63
1845.0      56
1090.0      54
1052.0      53
1664.0      52
1154.0      52
1513.0      51
1065.0      50
1237.0      49
1440.0      46
1205.0      46
1119.0      46
1088.0      46
1265.0      45
1395.0      44
1666.0      43
1585.0      43
1209.0      43
1134.0      42
1365.0      42
1162.0      42
1288.0      40
1248.0      40
1393.0      40
1087.0      39
1092.0    

In [ ]:
df["Weight_kg"].fillna("?", inplace=True)

In [ ]:
df.groupby(["make_model", "body_type","Weight_kg"]).price.describe()

count          mean           std  \
make_model     body_type     Weight_kg                                      
Audi A1        Compact       102.0        1.0  19229.000000           NaN   
                             1010.0       2.0  15450.000000    707.106781   
                             1035.0       6.0  16796.666667   2617.866816   
                             1040.0       2.0  20424.500000   2933.786035   
                             1065.0      36.0  20971.777778   1982.553564   
                             1090.0       9.0  21059.222222   2945.232919   
                             1100.0       2.0  27525.000000   1732.411614   
                             1105.0      12.0  13629.166667    228.093216   
                             1110.0      22.0  14322.000000   1738.096633   
                             1115.0       3.0  20833.333333   1950.854514   
                             1125.0       1.0  13945.000000           NaN   
                             1133.0       3.0  21223.333333    251.661148   
                             1134.0       6.0  16258.333333    988.137980   
                             1135.0      53.0  16554.679245   2170.538756   
                             1138.0       1.0  22140.000000           NaN   
                             1140.0       1.0  21940.000000           NaN   
                             1147.0       1.0  22990.000000           NaN   
                             1150.0       1.0  22980.000000           NaN   
                             1155.0       5.0  18533.000000   3476.373110   
                             1165.0      77.0  20264.259740   2560.316949   
                             1180.0      86.0  20753.616279   2638.046774   
                             1195.0      40.0  15930.725000   1308.218695   
                             1200.0      65.0  26139.553846   2769.602990   
                             1205.0       5.0  15098.800000   1223.635853   
                             1215.0       9.0  20182.555556   2365.728963   
                             1220.0       2.0  18090.000000   3733.523805   
                             1225.0       4.0  15887.750000   1550.406479   
                             1230.0       3.0  23250.000000   2655.654345   
                             1235.0       2.0  16590.000000    141.421356   
                             1240.0       7.0  25600.714286   3398.826303   
                             1250.0      17.0  16218.588235   3691.278370   
                             1275.0       5.0  20453.800000   4993.466952   
                             1280.0       1.0  25490.000000           NaN   
                             1285.0       3.0  15516.666667   1590.985020   
                             1335.0       3.0  28832.333333    237.563325   
                             1345.0       3.0  16106.666667    317.857410   
                             1485.0       5.0  15320.000000    408.656335   
                             1500.0       1.0  13500.000000           NaN   
                             1580.0       1.0  14280.000000           NaN   
                             1600.0       4.0  15550.000000    700.000000   
                             1625.0       1.0  15990.000000           NaN   
                             1675.0       1.0  11470.000000           NaN   
                             1792.0       1.0  13990.000000           NaN   
                             ?          526.0  18738.950570   4732.907318   
               Coupe         1010.0       1.0  13950.000000           NaN   
                             ?            1.0  15900.000000           NaN   
               Other         1165.0       1.0  21850.000000           NaN   
                             1205.0       1.0  23490.000000           NaN   
                             1230.0       1.0  25900.000000           NaN   
                             1250.0       1.0  16900.000000           NaN   
                             ?      

In [ ]:
df["Weight_kg"].replace("?", np.nan, inplace=True)

In [ ]:
fill(df, "make_model", "body_type", "Weight_kg", "mode")

In [ ]:
df["Weight_kg"].value_counts(dropna=False)

1163.0    1580
1360.0    1402
1487.0     954
1135.0     828
1425.0     724
1180.0     694
1273.0     656
1165.0     577
1503.0     570
1734.0     529
1087.0     291
1335.0     242
1365.0     211
1199.0     205
1350.0     156
1119.0     153
1355.0     136
1280.0     127
1260.0     127
1278.0     123
1275.0     112
1255.0     108
1200.0     107
1522.0     103
1659.0     102
1195.0      96
1120.0      93
1403.0      91
1701.0      87
1250.0      85
1685.0      83
1441.0      82
1285.0      80
1308.0      80
1090.0      77
1110.0      75
1613.0      75
1279.0      72
1364.0      70
1345.0      67
1733.0      65
1141.0      64
1071.0      64
1325.0      64
1209.0      64
1230.0      63
1205.0      59
1845.0      56
1052.0      53
1154.0      52
1664.0      52
NaN         52
1513.0      51
1065.0      50
1237.0      49
1088.0      46
1440.0      46
1265.0      45
1395.0      44
1666.0      43
1585.0      43
1134.0      42
1162.0      42
1248.0      40
1288.0      40
1393.0      40
1092.0    

# Drive chain-58 NaN

In [ ]:
df["Drive_chain"].value_counts(dropna=False)

front    8886
NaN      6858
4WD       171
rear        4
Name: Drive_chain, dtype: int64

In [ ]:
df["Drive_chain"].fillna("?", inplace=True)

In [ ]:
df.groupby(["make_model", "body_type", "Drive_chain"]).price.describe()

count          mean           std  \
make_model     body_type     Drive_chain                                       
Audi A1        Compact       4WD             2.0  14790.000000   1258.650071   
                             ?             352.0  17620.869318   4226.116239   
                             front         685.0  20008.223358   4511.347852   
               Coupe         ?               2.0  14925.000000   1378.858223   
               Other         ?               1.0  23490.000000           NaN   
                             front          12.0  21904.166667   3535.704786   
               Sedans        4WD             1.0  15450.000000           NaN   
                             ?             560.0  17820.333929   4359.643168   
                             front         977.0  19099.766633   4442.664502   
               Station wagon ?               3.0  24593.000000   7537.216131   
                             front          18.0  16681.111111   2493.673215   
               Van           front           1.0  29000.000000           NaN   
Audi A2        Off-Road      front           1.0  28200.000000           NaN   
Audi A3        Compact       4WD             1.0  67600.000000           NaN   
                             ?              25.0  18236.520000   3427.235865   
                             front         156.0  18839.365385   6078.743443   
               Convertible   4WD             1.0  56100.000000           NaN   
                             ?               4.0  26112.500000   6123.775388   
                             front           3.0  20897.666667  10003.500204   
               Coupe         ?               1.0  16990.000000           NaN   
                             front           3.0  17996.666667   1005.004146   
               Other         ?               6.0  17856.666667   5539.031203   
                             front          10.0  21907.000000   3249.338565   
               Sedans        4WD            17.0  32913.764706  20353.177865   
                             ?             878.0  20828.750569   5863.036376   
                             front        1703.0  21219.399295   3549.137668   
               Station wagon ?             149.0  20254.744966   5473.076587   
                             front         133.0  21165.646617   3552.001387   
Opel Astra     Compact       ?              81.0  17993.345679   6740.198121   
                             front         104.0  14802.836538   5080.909937   
               Coupe         ?               2.0  22240.000000  10960.155108   
               Off-Road      front           1.0  18590.000000           NaN   
               Other         ?              62.0  13857.645161   2413.144341   
                             front           5.0  13947.000000   2986.493429   
               Sedans        4WD             2.0  13075.000000   3641.599923   
                             ?             507.0  15488.443787   4123.966335   
                             front         544.0  16416.242647   4600.631934   
               Station wagon ?             519.0  15056.554913   5131.009341   
                             front         692.0  16306.960983   5328.760761   
Opel Corsa     Compact       ?             599.0  11077.415693   2984.777005   
                             front         631.0  12471.678288   3020.674196   
               Coupe         ?               6.0   9198.333333   1848.463326   
                             front           7.0   8407.142857   1342.394804   
               Off-Road      ?               3.0   8166.666667    461.880215   
               Other         ?              77.0   9331.441558   1534.857232   
                             front          10.0  13069.900000   2779.689049   
               Sedans        ?             359.0   9755.080780   2483.223878   
                             front         516.0  10519.350775   2692.189289   
               Transporter   ?             

In [ ]:
cond = (df['make_model'] == "Renault Duster") & (df["body_type"] == "Off-Road")

In [ ]:
df.loc[cond,'Drive_chain'] = df.loc[cond,'Drive_chain'].replace('?','4WD')

In [ ]:
df["Drive_chain"].value_counts(dropna=False)

front    8886
?        6826
4WD       203
rear        4
Name: Drive_chain, dtype: int64

In [ ]:
df["Drive_chain"] = df["Drive_chain"].replace('?', np.nan)

In [ ]:
df["Drive_chain"].value_counts(dropna=False)

front    8886
NaN      6826
4WD       203
rear        4
Name: Drive_chain, dtype: int64

In [ ]:
fill(df, "make_model", "body_type", "Drive_chain", "mode")

In [ ]:
df["Drive_chain"].value_counts(dropna=False)

front    15653
4WD        204
NaN         58
rear         4
Name: Drive_chain, dtype: int64

# CO2 Emission

In [ ]:
df["CO2_Emission"].value_counts(dropna=False)

NaN        2436
120.000     740
99.000      545
97.000      537
104.000     501
10.000      477
103.000     445
114.000     382
124.000     372
107.000     362
108.000     362
119.000     361
106.000     349
128.000     329
126.000     282
85.000      275
118.000     270
110.000     266
127.000     257
117.000     254
111.000     237
113.000     235
109.000     234
139.000     224
140.000     218
129.000     213
135.000     202
105.000     202
9.000       198
130.000     180
123.000     178
150.000     174
11.000      171
143.000     171
95.000      161
116.000     157
141.000     156
98.000      151
133.000     145
136.000     145
137.000     133
125.000     132
134.000     130
145.000     126
149.000     117
153.000     113
147.000     109
101.000     105
13.000      100
115.000      86
1.000        84
121.000      82
138.000      75
93.000       66
14.000       59
168.000      58
131.000      48
148.000      48
154.000      40
144.000      40
94.000       37
100.000      36
146.000 

In [ ]:
df["CO2_Emission"].fillna("?", inplace=True)

In [ ]:
df.groupby(["make_model", "body_type","CO2_Emission"]).price.describe()

count          mean           std  \
make_model     body_type     CO2_Emission                                      
Audi A1        Compact       1.0             1.0  20881.000000           NaN   
                             9.0             1.0  12479.000000           NaN   
                             10.0          147.0  16946.870748   2850.063119   
                             11.0           25.0  19960.360000   2676.261601   
                             14.0            2.0  28750.000000    268.700577   
                             90.0            1.0  12880.000000           NaN   
                             91.0            5.0  13038.000000   1108.927410   
                             94.0            5.0  13960.000000   1232.071427   
                             97.0          113.0  15449.592920   2445.754651   
                             98.0           33.0  16574.363636   2637.086518   
                             99.0           19.0  16043.947368   2257.402993   
                             100.0           4.0  16127.500000   1583.400876   
                             103.0          30.0  21233.033333   1794.550611   
                             104.0         126.0  17411.412698   3149.372642   
                             105.0          14.0  22234.857143    812.264248   
                             106.0          43.0  19831.790698   3335.793270   
                             107.0          13.0  16527.692308   1620.412262   
                             108.0          42.0  23587.095238   3375.692513   
                             109.0          34.0  17029.323529   2514.314097   
                             110.0          51.0  26340.372549   2434.737981   
                             111.0         138.0  23765.282609   3477.921290   
                             113.0           5.0  20388.000000   2796.188477   
                             114.0           1.0  15490.000000           NaN   
                             115.0           7.0  24067.857143   6950.649764   
                             116.0          20.0  23580.950000   2813.102847   
                             117.0           8.0  19094.875000   2942.954077   
                             118.0           2.0  13615.000000     91.923882   
                             119.0           1.0  12550.000000           NaN   
                             120.0          27.0  15850.000000      0.000000   
                             121.0           2.0  18710.000000   3224.406922   
                             123.0          17.0  19308.647059   3816.070049   
                             126.0           1.0  19999.000000           NaN   
                             127.0           2.0  19888.000000      0.000000   
                             134.0           2.0  22495.000000    841.457070   
                             136.0           2.0  29088.000000    128.693434   
                             137.0           2.0  28660.000000    282.842712   
                             331.0           1.0  12990.000000           NaN   
                             ?              92.0  18124.228261   4461.867189   
               Coupe         97.0            1.0  13950.000000           NaN   
                             ?               1.0  15900.000000           NaN   
               Other         97.0            1.0  16590.000000           NaN   
                             99.0            2.0  16900.000000      0.000000   
                             106.0           1.0  21850.000000           NaN   
                             108.0           3.0  26233.333333    577.350269   
                             134.0           1.0  23490.000000           NaN   
                             ?               5.0  22382.000000    536.954374   
               Sedans        9.0            42.0  15764.547619   1578.276402   
                             10.0          198.0  17497.909091   2518.189324   
                             11.0          

In [ ]:
df["CO2_Emission"].replace("?", np.nan, inplace=True)

In [ ]:
fill(df, "make_model", "body_type", "CO2_Emission", "median")

In [ ]:
df.drop("CO2_Emission", axis=1, inplace=True)

# Emission_Class 

In [ ]:
df["Emission_Class"].value_counts(dropna=False)

Euro 6    12173
NaN        3628
Euro 5       78
Euro 4       40
Name: Emission_Class, dtype: int64

In [ ]:
df["Emission_Class"].fillna("?", inplace=True)

In [ ]:
df["Emission_Class"].value_counts(dropna=False)

Euro 6    12173
?          3628
Euro 5       78
Euro 4       40
Name: Emission_Class, dtype: int64

In [ ]:
df.groupby(["make_model", "age", "Fuel", "Emission_Class"]).price.describe()

count          mean           std  \
make_model     age Fuel     Emission_Class                                      
Audi A1        0.0 Benzine  ?               211.0  24331.312796   2693.675149   
                            Euro 6          271.0  24241.188192   3417.536836   
                   Diesel   Euro 6            1.0  17900.000000           NaN   
               1.0 Benzine  ?                98.0  19334.581633   2864.664450   
                            Euro 5            3.0  17862.666667   1840.592658   
                            Euro 6          334.0  18793.059880   2829.853482   
                   Diesel   ?                14.0  19012.000000   2408.027057   
                            Euro 5            1.0  16800.000000           NaN   
                            Euro 6          294.0  17762.275510   2063.185255   
               2.0 Benzine  ?                57.0  17858.070175   2525.660266   
                            Euro 5            1.0  10999.000000           NaN   
                            Euro 6          213.0  16718.394366   2172.885243   
                   Diesel   ?                29.0  15002.206897   1467.289128   
                            Euro 6          132.0  16343.378788   1851.917960   
               3.0 Benzine  ?                58.0  14752.706897   2108.494691   
                            Euro 4            1.0  12990.000000           NaN   
                            Euro 6          260.0  14495.550000   2027.212528   
                   Diesel   ?                35.0  14612.085714   1958.519104   
                            Euro 5            4.0  14210.000000   1478.309846   
                            Euro 6          271.0  14421.815498   1699.495242   
Audi A2        1.0 Diesel   Euro 6            1.0  28200.000000           NaN   
Audi A3        0.0 Benzine  ?                82.0  26124.682927   4123.771984   
                            Euro 6          165.0  25432.418182   4482.330244   
                   Diesel   ?               106.0  25242.292453   1869.126381   
                            Euro 6          163.0  25051.865031   1833.831150   
                   LPG/CNG  ?                 1.0  24300.000000           NaN   
                            Euro 6            2.0  28450.000000   2192.031022   
               1.0 Benzine  ?                33.0  22359.303030   3664.601986   
                            Euro 6          119.0  25687.663866  11646.908841   
                   Diesel   ?                31.0  21614.354839   1609.743034   
                            Euro 5            1.0  20500.000000           NaN   
                            Euro 6          590.0  21844.547458   2194.096916   
                   Electric Euro 6            3.0  37449.666667   1361.635169   
               2.0 Benzine  ?                17.0  20045.235294   3616.920004   
                            Euro 6          125.0  20985.752000   6870.601898   
                   Diesel   ?                65.0  18335.600000   1433.653417   
                            Euro 6          462.0  19844.153680   3033.709820   
                   LPG/CNG  Euro 6            6.0  22566.666667   2001.665973   
               3.0 Benzine  ?                42.0  15427.880952   1135.562313   
                            Euro 5            1.0  14550.000000           NaN   
                            Euro 6           99.0  16457.121212   4422.134828   
                   Diesel   ?                68.0  15673.750000   1530.018870   
                            Euro 5            5.0  16698.000000   2111.520779   
                            Euro 6          581.0  16544.117040   2580.727616   
                   Electric Euro 6            1.0  24950.000000           NaN   
                   LPG/CNG  ?                 5.0  18220.000000    156.524758   
                            Euro 6           16.0  18749.937500   2688.004228   
Opel Astra     0.0 Benzine  ?               178.0  21603.696629   5389.3439

In [ ]:
df["Emission_Class"].replace("?", np.nan, inplace=True)

In [ ]:
df["Emission_Class"].value_counts(dropna=False)

Euro 6    12173
NaN        3628
Euro 5       78
Euro 4       40
Name: Emission_Class, dtype: int64

In [ ]:
fill(df, "age", "Fuel", "Emission_Class", "ffill")

In [ ]:
df.drop("Emission_Class", axis=1, inplace=True)

# Comfort_Convenience

In [ ]:
df["Comfort_Convenience"].value_counts(dropna=False).to_frame()

,Comfort_Convenience
NaN,920
"Air conditioning,Electrical side mirrors,Hill Holder,Power windows",216
"Air conditioning,Electrical side mirrors,Power windows",134
"Air conditioning,Power windows",130
"Air conditioning,Armrest,Automatic climate control,Cruise control,Electrical side mirrors,Leather steering wheel,Light sensor,Lumbar support,Multi-function steering wheel,Navigation system,Park Distance Control,Parking assist system sensors front,Parking assist system sensors rear,Power windows,Rain sensor,Seat heating,Start-stop system",105
...,...
"Air conditioning,Armrest,Cruise control,Electrical side mirrors,Navigation system,Park Distance Control,Parking assist system sensors front,Parking assist system sensors rear,Power windows,Seat heating",1
"Air conditioning,Automatic climate control,Cruise control,Electrical side mirrors,Heated steering wheel,Leather steering wheel,Light sensor,Multi-function steering wheel,Navigation system,Park Distance Control,Parking assist system sensors front,Parking assist system sensors rear,Power windows,Rain sensor,Seat heating",1
"Air conditioning,Armrest,Automatic climate control,Cruise control,Hill Holder,Leather steering wheel,Multi-function steering wheel,Navigation system,Park Distance Control,Parking assist system sensors front,Parking assist system sensors rear,Power windows",1
"Air conditioning,Armrest,Automatic climate control,Cruise control,Electrical side mirrors,Hill Holder,Leather steering wheel,Light sensor,Multi-function steering wheel,Power windows,Rain sensor,Start-stop system",1


In [ ]:
fill(df, "make_model", "body_type", "Comfort_Convenience", "mode")

In [ ]:
df["Comfort_Convenience"].value_counts(dropna=False).to_frame()

,Comfort_Convenience
"Air conditioning,Electrical side mirrors,Hill Holder,Power windows",315
"Air conditioning,Armrest,Automatic climate control,Cruise control,Electrical side mirrors,Leather steering wheel,Light sensor,Lumbar support,Multi-function steering wheel,Navigation system,Park Distance Control,Parking assist system sensors front,Parking assist system sensors rear,Power windows,Rain sensor,Seat heating,Start-stop system",187
"Air conditioning,Automatic climate control,Cruise control,Multi-function steering wheel,Park Distance Control,Power windows",158
"Air conditioning,Electrical side mirrors,Power windows",134
"Air conditioning,Power windows",130
...,...
"Air conditioning,Automatic climate control,Cruise control,Electrical side mirrors,Heated steering wheel,Leather steering wheel,Light sensor,Multi-function steering wheel,Navigation system,Park Distance Control,Parking assist system sensors front,Parking assist system sensors rear,Power windows,Rain sensor,Seat heating",1
"Air conditioning,Armrest,Automatic climate control,Cruise control,Hill Holder,Leather steering wheel,Multi-function steering wheel,Navigation system,Park Distance Control,Parking assist system sensors front,Parking assist system sensors rear,Power windows",1
"Air conditioning,Armrest,Automatic climate control,Cruise control,Electrical side mirrors,Hill Holder,Leather steering wheel,Light sensor,Multi-function steering wheel,Power windows,Rain sensor,Start-stop system",1
"Air conditioning,Armrest,Automatic climate control,Cruise control,Hill Holder,Leather steering wheel,Light sensor,Lumbar support,Multi-function steering wheel,Navigation system,Park Distance Control,Parking assist system sensors rear,Power windows,Rain sensor,Start-stop system,Tinted windows",1


# Entertainment_Media 36 NaN

In [ ]:
df["Entertainment_Media"].value_counts(dropna=False).to_frame()

,Entertainment_Media
NaN,1374
"Bluetooth,Hands-free equipment,On-board computer,Radio,USB",1282
"Bluetooth,Hands-free equipment,MP3,On-board computer,Radio,USB",982
"Bluetooth,CD player,Hands-free equipment,MP3,On-board computer,Radio,USB",783
"On-board computer,Radio",487
Radio,477
"Bluetooth,CD player,Hands-free equipment,On-board computer,Radio,USB",465
On-board computer,461
"Bluetooth,CD player,Hands-free equipment,MP3,On-board computer,Radio",404
"Bluetooth,On-board computer,Radio",397


In [ ]:
fill(df, "make_model", "body_type", "Entertainment_Media", "mode")

In [ ]:
df["Entertainment_Media"].value_counts(dropna=False).to_frame()

,Entertainment_Media
"Bluetooth,Hands-free equipment,On-board computer,Radio,USB",1629
"Bluetooth,Hands-free equipment,MP3,On-board computer,Radio,USB",1063
"Bluetooth,CD player,Hands-free equipment,MP3,On-board computer,Radio,USB",915
On-board computer,678
Radio,558
"Bluetooth,Hands-free equipment,On-board computer,Radio",494
"On-board computer,Radio",487
"Bluetooth,CD player,Hands-free equipment,On-board computer,Radio,USB",466
"Bluetooth,On-board computer,Radio",427
"Bluetooth,CD player,Hands-free equipment,MP3,On-board computer,Radio",404


# Extras-44NaN

In [ ]:
df["Extras"].value_counts(dropna=False).to_frame()

,Extras
Alloy wheels,3245
NaN,2962
"Alloy wheels,Touch screen",697
"Alloy wheels,Voice Control",577
"Alloy wheels,Touch screen,Voice Control",541
"Alloy wheels,Roof rack",385
"Alloy wheels,Sport seats",380
Roof rack,337
"Alloy wheels,Catalytic Converter",271
"Alloy wheels,Sport seats,Sport suspension",234


In [ ]:
fill(df, "make_model", "body_type", "Extras", "mode")

In [ ]:
df["Extras"].value_counts(dropna=False).to_frame()

,Extras
Alloy wheels,5850
"Alloy wheels,Touch screen",697
"Alloy wheels,Voice Control",582
"Alloy wheels,Touch screen,Voice Control",544
Roof rack,531
"Alloy wheels,Roof rack",484
"Alloy wheels,Sport seats",382
"Alloy wheels,Catalytic Converter",275
"Alloy wheels,Sport seats,Sport suspension",234
"Alloy wheels,Catalytic Converter,Touch screen",228


# Safety_Security

In [ ]:
df["Safety_Security"].value_counts(dropna=False).to_frame()

,Safety_Security
NaN,982
"ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control",538
"ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control",480
"ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control,Xenon headlights",275
"ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,LED Daytime Running Lights,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control",272
...,...
"ABS,Adaptive Cruise Control,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Emergency brake assistant,Immobilizer,Isofix,LED Daytime Running Lights,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control",1
"ABS,Adaptive Cruise Control,Blind spot monitor,Central door lock,Central door lock with remote control,Daytime running lights,Driver-side airbag,Electronic stability control,Emergency brake assistant,Fog lights,Isofix,Lane departure warning system,LED Daytime Running Lights,LED Headlights,Night view assist,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control,Traffic sign recognition",1
"ABS,Adaptive Cruise Control,Alarm system,Central door lock,Central door lock with remote control,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Head airbag,Immobilizer,Isofix,LED Daytime Running Lights,Passenger-side airbag,Power steering,Rear airbag,Side airbag,Traction control",1
"ABS,Alarm system,Central door lock,Daytime running lights,Driver-side airbag,Fog lights,Passenger-side airbag,Power steering",1


In [ ]:
fill(df, "make_model", "body_type", "Safety_Security", "mode")

In [ ]:
df["Safety_Security"].value_counts(dropna=False).to_frame()

,Safety_Security
"ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control",729
"ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control",480
"ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Immobilizer,Isofix,LED Daytime Running Lights,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control",372
"ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Immobilizer,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control,Xenon headlights",351
"ABS,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Isofix,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control",306
...,...
"ABS,Adaptive Cruise Control,Central door lock,Daytime running lights,Driver-side airbag,Electronic stability control,Emergency brake assistant,Immobilizer,Isofix,LED Daytime Running Lights,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control",1
"ABS,Adaptive Cruise Control,Blind spot monitor,Central door lock,Central door lock with remote control,Daytime running lights,Driver-side airbag,Electronic stability control,Emergency brake assistant,Fog lights,Isofix,Lane departure warning system,LED Daytime Running Lights,LED Headlights,Night view assist,Passenger-side airbag,Power steering,Side airbag,Tire pressure monitoring system,Traction control,Traffic sign recognition",1
"ABS,Adaptive Cruise Control,Alarm system,Central door lock,Central door lock with remote control,Daytime running lights,Driver-side airbag,Electronic stability control,Fog lights,Head airbag,Immobilizer,Isofix,LED Daytime Running Lights,Passenger-side airbag,Power steering,Rear airbag,Side airbag,Traction control",1
"ABS,Alarm system,Central door lock,Daytime running lights,Driver-side airbag,Fog lights,Passenger-side airbag,Power steering",1


# Gears- Gearing Type olmadığı için çalışmıyor.

In [ ]:
df["Gears"].value_counts(dropna=False)

6.0     5822
NaN     4712
5.0     3239
7.0     1908
8.0      224
9.0        6
4.0        2
1.0        2
3.0        2
50.0       1
2.0        1
Name: Gears, dtype: int64

In [ ]:
df["Gears"].fillna("?", inplace=True)

In [ ]:
df["Gears"].value_counts(dropna=False)

6.0     5822
?       4712
5.0     3239
7.0     1908
8.0      224
9.0        6
1.0        2
3.0        2
4.0        2
2.0        1
50.0       1
Name: Gears, dtype: int64

In [ ]:
df.groupby(["make_model", "body_type", "Gearing_Type", "Gears"]).price.describe()

KeyError: 'Gearing_Type'

In [ ]:
df["Gears"].replace([1,2,3,4,9,50,"-"], np.nan, inplace=True)

In [ ]:
df["Gears"].value_counts(dropna=False)

6.0    5822
?      4712
5.0    3239
7.0    1908
8.0     224
NaN      14
Name: Gears, dtype: int64

In [ ]:
for group1 in list(df["make_model"].unique()):
    for group2 in list(df["body_type"].unique()):
        for group3 in list(df["Gearing_Type"].unique()):
            cond1 = df["make_model"]==group1
            cond2 = (df["make_model"]==group1) & (df["body_type"]==group2)
            cond3 = (df["make_model"]==group1) & (df["body_type"]==group2) & (df["Gearing_Type"]==group3)
            mode1 = list(df[cond1]["Gears"].mode())
            mode2 = list(df[cond2]["Gears"].mode())
            mode3 = list(df[cond3]["Gears"].mode())
            if mode3 != []:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df[cond3]["Gears"].mode()[0])
            elif mode2 != []:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df[cond2]["Gears"].mode()[0])
            elif mode1 != []:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df[cond1]["Gears"].mode()[0])
            else:
                df.loc[cond3, "Gears"] = df.loc[cond3, "Gears"].fillna(df["Gears"].mode()[0])

KeyError: 'Gearing_Type'

In [ ]:
df["Gears"].value_counts(dropna=False)

6.0    5822
?      4712
5.0    3239
7.0    1908
8.0     224
NaN      14
Name: Gears, dtype: int64

# Cons_Comb- Çalışmadı

In [ ]:
df["Cons_Comb"].value_counts(dropna=False)

NaN     2033
5.4      770
3.9      733
4.0      713
5.1      657
4.4      623
5.6      618
4.7      602
3.8      585
4.8      546
5.0      545
4.5      523
5.2      454
4.2      435
4.6      426
4.9      393
5.3      380
5.5      380
5.9      369
3.7      369
5.7      342
4.1      342
6.0      331
4.3      307
3.3      307
3.5      288
6.2      216
3.6      194
6.3      181
6.1      175
5.8      164
6.6      148
6.8      136
3.4      106
6.4       75
3.0       69
7.4       66
6.7       43
6.5       43
7.1       38
10.0      34
6.9       27
3.2       25
8.3       20
7.6       14
7.0       10
3.1        7
7.2        6
7.8        6
8.0        5
51.0       4
8.6        4
8.7        3
1.6        3
7.9        3
38.0       2
0.0        2
40.0       2
7.3        2
8.1        2
32.0       1
1.2        1
33.0       1
50.0       1
11.0       1
55.0       1
54.0       1
1.0        1
9.1        1
7.5        1
43.0       1
46.0       1
13.8       1
Name: Cons_Comb, dtype: int64

In [ ]:
cons_comb = (df["Cons_Country"] + df["Cons_City"])/2

In [ ]:
df["Cons_Comb"] = df["Cons_Comb"].fillna(cons_comb)

In [ ]:
df["Cons_Comb"].fillna("?", inplace=True)

In [ ]:
df.groupby(["make_model", "body_type","Cons_Comb"]).price.describe()

count          mean  \
make_model     body_type     Cons_Comb                                 
Audi A1        Compact       3.0                   4.0  14370.250000   
                             3.4                   6.0  12755.000000   
                             3.5                   3.0  14613.000000   
                             3.6                   5.0  13960.000000   
                             3.7                  50.0  15902.200000   
                             3.8                  18.0  16158.055556   
                             3.9                  41.0  15771.902439   
                             4.0                  79.0  16795.670886   
                             4.1                  38.0  16540.736842   
                             4.2                  87.0  15631.126437   
                             4.3                  17.0  16628.705882   
                             4.4                 193.0  17618.922280   
                             4.5                  11.0  21482.909091   
                             4.6                  52.0  20729.961538   
                             4.7                  33.0  22102.666667   
                             4.8                  93.0  25162.548387   
                             4.9                 176.0  23133.289773   
                             5.0                   9.0  21739.444444   
                             5.1                  50.0  18187.280000   
                             5.2                  18.0  19429.833333   
                             5.6                   2.0  19888.000000   
                             5.8                   2.0  22495.000000   
                             6.0                   8.0  28797.125000   
                             13.8                  1.0  12990.000000   
                             ?                    43.0  16765.790698   
               Coupe         4.2                   1.0  13950.000000   
                             ?                     1.0  15900.000000   
               Other         3.8                   2.0  16900.000000   
                             4.2                   1.0  16590.000000   
                             4.7                   1.0  21850.000000   
                             4.8                   3.0  26233.333333   
                             5.8                   1.0  23490.000000   
                             ?                     5.0  22382.000000   
               Sedans        3.0                   1.0  15450.000000   
                             3.4                  20.0  14829.400000   
                             3.5                  48.0  15679.125000   
                             3.6                  13.0  15590.000000   
                             3.7                 140.0  15242.100000   
                             3.8                  93.0  16119.021505   
                             3.9                  93.0  16511.709677   
                             4.0                  81.0  19200.000000   
                             4.1                  15.0  16189.333333   
                             4.2                 219.0  14805.858447   
                             4.3                  43.0  16173.558140   
                             4.4                 156.0  17887.403846   
                             4.5                   8.0  19561.000000   
                             4.55                  1.0  11990.000000   
                             4.6                  61.0  20956.196721   
                             4.7                   8.0  20377.750000   
                             4.8                 254.0  25234.519685   
                             4.9                  84.0  21241.690476   
                             5.0                   7.0  23258.571429   
                             5.1                  27.0  16992.074074   
                             5.2                   5.0  20150.200000   
                 

In [ ]:
df["Cons_Comb"].replace([0.0, 1.0, 1.2, 1.6, 10, 11, 13.8, 32.0, 33.0, 38.0, 40.0, 43.0, 46.0, 50.0, 51.0, 54.0, 55.0, "-"], np.nan, inplace=True)

In [ ]:
df["Cons_Comb"].value_counts(dropna=False)

?                     1925
5.4                    770
3.9                    733
4.0                    713
5.1                    657
4.4                    623
5.6                    618
4.7                    607
3.8                    585
4.8                    546
5.0                    545
4.5                    524
5.2                    454
4.2                    435
4.6                    426
4.9                    393
5.3                    380
5.5                    380
5.9                    369
3.7                    369
4.1                    342
5.7                    342
6.0                    331
4.3                    307
3.3                    307
3.5                    288
6.2                    216
3.6                    194
6.3                    181
6.1                    175
5.8                    165
6.6                    148
6.8                    136
3.4                    125
6.4                     75
3.0                     69
7.4                     66
N

In [ ]:
fill(df, "make_model", "body_type", "Cons_Comb", "median")

TypeError: 

# Cons_City

In [ ]:
df.drop("Cons_City", axis = 1, inplace = True)

# Cons_Country

In [ ]:
df.drop("Cons_Country", axis = 1, inplace = True)

In [ ]:
df.shape

(15919, 23)

In [ ]:
df.isnull().sum()/df.shape[0]*100

make_model              0.000000
body_type               0.376908
price                   0.000000
vat                    28.349771
km                      6.432565
Type                    0.012564
Fuel                    0.000000
Comfort_Convenience     0.100509
Entertainment_Media     0.226145
Extras                  0.276399
Safety_Security         0.094227
Gears                   0.087945
age                    10.032037
Previous_Owners        41.711163
hp_kW                   0.552799
Inspection_new          0.000000
Upholstery_type         0.000000
Nr_of_Doors             0.006282
Nr_of_Seats             0.295245
Displacement_cc         0.144481
Weight_kg               0.326654
Drive_chain             0.364344
Cons_Comb               0.364344
dtype: float64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15919 entries, 0 to 15918
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   make_model           15919 non-null  object 
 1   body_type            15859 non-null  object 
 2   price                15919 non-null  int64  
 3   vat                  11406 non-null  object 
 4   km                   14895 non-null  float64
 5   Type                 15917 non-null  object 
 6   Fuel                 15919 non-null  object 
 7   Comfort_Convenience  15903 non-null  object 
 8   Entertainment_Media  15883 non-null  object 
 9   Extras               15875 non-null  object 
 10  Safety_Security      15904 non-null  object 
 11  Gears                15905 non-null  object 
 12  age                  14322 non-null  float64
 13  Previous_Owners      9279 non-null   float64
 14  hp_kW                15831 non-null  float64
 15  Inspection_new       15919 non-null 

In [94]:
df.to_csv("EDA_scout_car_filled_data.csv", index=False)